In [1]:
import pandas as pd
import pycountry

def clean_data(df):
    # Convert country names in 'region' to alpha-3 codes
    def convert_to_alpha_3(country_name):
        try:
            return pycountry.countries.lookup(country_name).alpha_3
        except LookupError:
            return None
    df['region'] = df['region'].apply(convert_to_alpha_3)
    # Convert the 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])
    # Create a new column 'ID' by splitting the 'link' column and taking the last part
    df['ID'] = df['url'].apply(lambda x: x.split('/')[-1])
    #drop useless tables
    df.drop(columns=['url', 'trend', 'rank', 'chart'], inplace=True)
    #add IsChristmasSong
    top_xmas_songs_df = pd.read_csv('data/top_xmas_songs_with_ids.csv')
    # Create a new column 'IsChristmasSong' and set it to True if the 'ID' is in the top Christmas songs DataFrame
    df['IsChristmasSong'] = df['ID'].isin(top_xmas_songs_df['SongID'])
    # Create a new column 'IsChristmasSongByTitle' and set it to True if 'christmas' is in the 'title' column
    df['IsChristmasSongByTitle'] = df['title'].str.lower().str.contains('christmas')
    # # Update 'IsChristmasSong' to True if either 'IsChristmasSong' or 'IsChristmasSongByTitle' is True
    df['IsChristmasSong'] = df['IsChristmasSong'] | df['IsChristmasSongByTitle']
    # Aggregate streams on ID, keep title and artist
    df = df[df['IsChristmasSong'] == True]
    # df = df.groupby(['ID', 'title', 'artist'], as_index=False)['streams'].sum()
    # df = df.sort_values(by=['streams'], ascending=[False]).reset_index(drop=True)
    # Group by region and date, sum streams
    df = df.groupby(['region', 'date'])['streams'].sum().reset_index()
    #add calendar week
    df['calendar_week'] = df['date'].dt.isocalendar().week
    # Group by calendar week and average all columns
    df = df.drop(columns=['date'])
    # Group by calendar_week and region, average streams
    df = df.groupby(['calendar_week', 'region'], as_index=False)['streams'].mean().round(0)
    # Pivot calendar_week values to columns and sort
    df = df.pivot(index='region', columns='calendar_week', values='streams')#.reset_index()
    # Add rows for column 1 and 53 into column 1, then drop 53
    df[1] = df[1] + df[53]
    df = df.drop(columns=[53])
    df = df.sort_values(by=[35,36,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,1,2,3,4,5,6,7,8,9,10], ascending=False)
    #scale based on countries max streams
    df = df.div(df.max(axis=1), axis=0)

    df.to_csv('data/transposed_avg_xmas_songs_weekly_streams.csv')
    df.to_json('data/transposed_avg_xmas_songs_weekly_streams.json', orient='columns')

    return df

# Loaded variable 'df' from URI: f:\Daten\Privat\Projekte\#01_Active_Projects\14_DJ_WS2024_Music-Scraper\daily_charts.csv
df = pd.read_csv(r'f:\Daten\Privat\Projekte\#01_Active_Projects\14_DJ_WS2024_Music-Scraper\daily_charts.csv')

df_clean = clean_data(df.copy())
df_clean.head()

calendar_week,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
region,,,,,,,,,,,,,,,,,,,,,
PHL,0.503059,0.206558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.03148,0.039319,0.046744,0.064136,0.126917,0.222899,0.321224,0.640611,1.000000
ARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.015668,0.030258,0.079009,0.122168,0.261969,0.407144,0.879376,1.000000
UKR,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.051878,0.134415,0.234082,0.349685,0.424336,0.624484
THA,0.212681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.08736,NaN,NaN,0.061968,0.107141,0.231155,0.279287,0.323691,1.000000
TWN,0.179788,0.042906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,0.044308,0.103437,0.226652,0.314009,0.535924,1.000000
